In [2]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [3]:
df = ssql("""
SELECT t1.distinct_id,
       datediff(now(), t2."注册时间") as "注册距今天数",
       t2.reg_gender
FROM
  (SELECT distinct_id
   FROM events
   WHERE event = 'pay'
     AND date BETWEEN '2020-10-16' AND current_date()
     AND appId IN ('20014',
                   '30015')
   GROUP BY 1)t1
JOIN
  (SELECT second_id,
          FROM_UNIXTIME(CAST($signup_time/1000 AS BIGINT))AS "注册时间",
          reg_gender
   FROM users
   WHERE second_id IS NOT NULL)t2 ON t1.distinct_id = t2.second_id
""")

In [4]:
df

,注册距今天数,distinct_id,reg_gender
0,2.0,115972941,f
1,4.0,119359586,f
2,2.0,131186778,m
3,1052.0,99839111,m
4,NaN,52843899,NaN
...,...,...,...
574,2.0,122136477,m
575,1.0,149443021,f
576,5.0,129702840,m
577,0.0,142487290,m


In [9]:
df['注册距今天数'].fillna(df['注册距今天数'].median(),inplace=True)

In [10]:
df

,注册距今天数,distinct_id,reg_gender
0,2.0,115972941,f
1,4.0,119359586,f
2,2.0,131186778,m
3,1052.0,99839111,m
4,3.0,52843899,NaN
...,...,...,...
574,2.0,122136477,m
575,1.0,149443021,f
576,5.0,129702840,m
577,0.0,142487290,m


In [11]:
df.describe()

,注册距今天数
count,579.000000
mean,121.991364
std,328.288803
min,0.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,1433.000000


In [17]:
pd.DataFrame(df['注册距今天数'].quantile([0,0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9,0.95,1.0])).T

,0.00,0.10,0.20,0.25,0.30,0.40,0.50,0.60,0.70,0.75,0.80,0.90,0.95,1.00
注册距今天数,0.0,1.0,1.0,1.0,2.0,2.0,3.0,4.0,4.0,5.0,6.0,419.4,1088.8,1433.0


In [18]:
df.groupby(by = 'reg_gender').count()

,注册距今天数,distinct_id
reg_gender,,
f,241,241
m,325,325


In [22]:
df_game = ssql("""
SELECT t1.distinct_id,
       t1.pay,
       t2."游戏局数",
       t2.duration
FROM
  (SELECT distinct_id,
          sum(pay) AS pay
   FROM events
   WHERE event = 'pay'
     AND date BETWEEN '2020-10-16' AND current_date()
     AND appId IN ('20014',
                   '30015')
   GROUP BY 1) t1
LEFT JOIN
  (SELECT distinct_id,
          count(distinct_id)AS "游戏局数",
          sum(duration/60000)AS duration
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date >='2020-10-16'
     AND time >'2020-10-16 12:00:00'
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id
""")

In [23]:
df_game

,duration,游戏局数,distinct_id,pay
0,238.009,34.0,139923409,94.0
1,440.586,72.0,123999663,6.0
2,917.918,132.0,122878916,6.0
3,24.327,5.0,118563664,88.0
4,545.449,121.0,112155576,6.0
...,...,...,...,...
588,NaN,NaN,117154871,6.0
589,NaN,NaN,128267381,6.0
590,NaN,NaN,124157923,6.0
591,NaN,NaN,72022499,6.0


In [24]:
df_game.to_clipboard()

In [32]:
pd.DataFrame(df_game['duration'].quantile([0,0.25,0.5,0.75,1])).T.to_clipboard()

In [34]:
pd.DataFrame(df_game['游戏局数'].quantile([0,0.25,0.5,0.75,1])).T.to_clipboard()

In [28]:
df_game_1 = ssql("""
SELECT distinct_id,
          count(distinct_id)AS "游戏局数",
          sum(duration/60000)AS duration
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date >='2020-10-16'
     AND time >'2020-10-16 12:00:00'
   GROUP BY 1""")

In [33]:
pd.DataFrame(df_game_1['duration'].quantile([0,0.25,0.5,0.75,1])).T.to_clipboard()

In [35]:
pd.DataFrame(df_game_1['游戏局数'].quantile([0,0.25,0.5,0.75,1])).T.to_clipboard()